# CSE-CIC-IDS 2017

In [1]:
import numpy as np
np.random.seed(42)
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import glob, time, os

import keras

Using TensorFlow backend.
C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Us

In [2]:
NOTEBOOK_PATH = "D:/Delta Stuff/Scifair20/"

In [3]:
f_names = glob.glob(NOTEBOOK_PATH + "IDS2017/MachineLearningCVE/*.csv")
df = pd.concat(tqdm((pd.read_csv(f) for f in f_names), total=len(f_names)))

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\concat.py:255: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  sort=sort,
 38%|███████████████████████████████▌                                                    | 3/8 [00:02<00:04,  1.02it/s]C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\concat.py:255: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  sort=sort,
100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:12<00:00,  1.52s/it]


In [4]:
df.reset_index(drop=True, inplace=True)

In [5]:
df.columns = list(map(lambda x: x.strip(), df.columns))

In [6]:
# Remove variables with no variance
remove_vars = []
for i in tqdm(range(len(df.columns))):
    col = df.columns[i]
    unique = df[col].unique()
    if len(unique) <= 1:
        remove_vars.append(col)

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:04<00:00, 19.08it/s]


In [7]:
remove_vars

['Bwd PSH Flags',
 'Bwd URG Flags',
 'Fwd Avg Bytes/Bulk',
 'Fwd Avg Packets/Bulk',
 'Fwd Avg Bulk Rate',
 'Bwd Avg Bytes/Bulk',
 'Bwd Avg Packets/Bulk',
 'Bwd Avg Bulk Rate']

In [8]:
df.drop(remove_vars, axis=1, inplace=True)

df.drop(["Flow Bytes/s", "Flow Packets/s"], axis='columns', inplace=True)

In [9]:
df.shape

(2830743, 69)

# Standardize data

In [10]:
from sklearn.preprocessing import PowerTransformer, MinMaxScaler

In [11]:
scaler = PowerTransformer(method='yeo-johnson', standardize=True)
# scaler = MinMaxScaler()

In [12]:
x_df = df.drop("Label", axis=1)
y_df = df.loc[:, "Label"]

In [13]:
x_scaled = scaler.fit_transform(x_df, y_df)

In [14]:
scaled_summary = pd.DataFrame(x_scaled, columns=x_df.columns).describe()
scaled_summary

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,...,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06
mean,5.614575e-17,-5.294087e-16,-1.481637e-15,-7.895848e-16,-6.103742e-16,4.526701e-16,-2.069120e-16,-5.198502e-16,-1.929056e-15,1.910080e-16,...,-6.024222e-18,-1.909197e-15,-2.522141e-16,8.514234e-18,2.377560e-17,1.824135e-16,8.731106e-17,-3.527785e-16,-2.910904e-16,1.681963e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-8.085491e+00,-2.286645e+01,-1.304010e+00,-1.655424e+00,-1.617208e+00,-1.349540e+00,-1.591983e+00,-9.953509e-01,-1.640853e+00,-7.155903e-01,...,-1.200386e+00,-2.752211e+01,-4.925726e-01,-2.797894e-01,-4.923377e-01,-4.926216e-01,-5.006239e-01,-2.968475e-01,-5.006238e-01,-5.006238e-01
25%,-8.738939e-01,-8.876095e-01,-2.338725e-01,-4.381973e-01,-5.559671e-01,-1.349540e+00,-6.855578e-01,-9.953509e-01,-6.405733e-01,-7.155903e-01,...,-1.200386e+00,-9.087109e-01,-4.925726e-01,-2.797894e-01,-4.923377e-01,-4.926216e-01,-5.006239e-01,-2.968475e-01,-5.006238e-01,-5.006238e-01
50%,-5.280598e-01,1.134869e-01,-2.338725e-01,1.225253e-01,1.168017e-01,1.793477e-01,1.106896e-01,-1.864637e-01,3.389061e-01,-7.155903e-01,...,1.163599e-01,-3.832149e-01,-4.925726e-01,-2.797894e-01,-4.923377e-01,-4.926216e-01,-5.006239e-01,-2.968475e-01,-5.006238e-01,-5.006238e-01
75%,5.271538e-01,9.319696e-01,8.888288e-01,7.029430e-01,5.919397e-01,5.791860e-01,4.752672e-01,1.140584e+00,5.905137e-01,1.240967e+00,...,6.579127e-01,8.493489e-01,-4.925726e-01,-2.797894e-01,-4.923377e-01,-4.926216e-01,-5.006239e-01,-2.968475e-01,-5.006238e-01,-5.006238e-01
max,1.713337e+00,1.538709e+00,2.168643e+00,3.041949e+00,5.883933e+00,4.035647e+00,3.233995e+00,2.329224e+00,4.685579e+00,1.704225e+00,...,2.511256e+00,3.492139e+01,2.093608e+00,3.576140e+00,2.098229e+00,2.092428e+00,2.006166e+00,3.369526e+00,2.006018e+00,2.006746e+00


In [15]:
y_df_group = y_df.copy()

for i in tqdm(range(len(y_df_group))):
    attack = y_df_group[i]
    group = attack
    if attack in ("DoS Hulk", "DoS GoldenEye", "DoS slowloris", "DoS Slowhttptest", "DDoS", "Heartbleed"):
        group = "DoS/DDoS"
    elif attack.startswith("Web Attack"):
        group = "Web Attack"
    elif attack in ("FTP-Patator", "SSH-Patator"):
        group = "Brute Force"
    elif attack == "Bot":
        group = "Botnet"
    
    
    y_df_group[i] = group

100%|█████████████████████████████████████████████████████████████████████| 2830743/2830743 [01:30<00:00, 31352.03it/s]


In [16]:
y_df.value_counts()

BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: Label, dtype: int64

In [17]:
y_df_group.value_counts()

BENIGN          2273097
DoS/DDoS         380699
PortScan         158930
Brute Force       13835
Web Attack         2180
Botnet             1966
Infiltration         36
Name: Label, dtype: int64

In [18]:
y_df_enc = pd.get_dummies(y_df_group)

In [19]:
x_scaled.shape

(2830743, 68)

In [20]:
pd.DataFrame(x_scaled, columns=x_df.columns).to_csv(NOTEBOOK_PATH + "IDS2017/x_scaled_powertransform.csv", index=False)

In [21]:
y_df_enc.to_csv(NOTEBOOK_PATH + "IDS2017/y_grouped_1henc.csv", index=False)